In [142]:
# Getting the input data
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats 
import shutil


In [143]:
# Specify the path where output.csv and ey_strain.csv is stored
# output.csv is csv file from loadframe which has data in the order Force, Displacement and Time
# ey_strain.csv is csv file from DIC software- ARAMIS or VIC which has data in the order Stage ( i.e time ) 
# and strain data

os.chdir('F:\P6')

# Reading all the data in csv format to the data frame

output = pd.read_csv("output.csv",skiprows = 2,names = ['Force','Disp','Time'])
output = output.apply(pd.to_numeric, errors ='coerce')
output = output.dropna(axis = 0)

eystrain = pd.read_csv("ey_strain.csv",skiprows = 3,names =['Stage','ey_strain'])
eystrain = eystrain.apply(pd.to_numeric, errors='coerce')
eystrain = eystrain.dropna(axis = 0)


print(output.head(5))



        Force      Disp      Time
0  118.696777  0.003383  0.868164
1  255.132233  0.008751  1.068359
2  343.885101  0.013570  1.268555
3  423.984711  0.018270  1.468750
4  488.151794  0.023793  1.668945


In [144]:
#Calculating Stress from the output data 

# Area of cross-section of tensile sample

Area = 22.107

# Changing strain from percentage to mm/mm format
eystrain["ey_strain"] = eystrain["ey_strain"]


# Calculating Stress from force 
output["stress"] = output["Force"]/Area

print(Area)

# Assembling all the data to a data frame
data = pd.concat([output['Time'], output['stress'], eystrain['Stage'], eystrain['ey_strain']],axis = 1, keys= ['time','stress','stage','strain'])




22.107


In [145]:
#Filtering stress

#Filtering out the Stress data corresponding to each Strain Stage

Stress = []
time = np.array(data['time'].dropna())
stage = np.array(data['stage'].dropna())
time_sorted = []

# Finding closest time value of stress corresponding to each strain stage data
for i in range(0, len(stage)):
    time_sorted.append(min(range(len(time)), key=lambda j: abs(time[j]-stage[i])))

for i in range (0, len(time_sorted)):
        Stress.append(data['stress'][time_sorted[i]]) 

Strain = np.array(data["strain"].dropna())
Stress = np.array(Stress)


print(len(Strain))
print(len(Stress))

Data = pd.DataFrame({'Strain':Strain,'Stress':Stress},columns = ['Strain','Stress'])


132
132


# Finding Split point 

In [146]:
def Split_Stress(stress, strain):
    
    # Split_point is strain at whaich standard deviation(ymodulus)/mean(ymodulus) > 10 %
    
    ymodulus = []
    
    window = 30
    
    for i in range(0,len(stress)-window):
        ymodulus.append((stress[i+window] - stress[i])/(strain[i+window] - strain[i]))
        m = np.mean(ymodulus)
        s = np.std(ymodulus)
        p = (s/m)*100
        if np.floor(p) > 15:
            global Split_Point
            Split_Point = strain[i]  
            break

    #Calculating Young's Modulus from raw data ey_strain

    #Filtering out the linear potion of the ey_strain and stress data to find the slope

    linear_strain = []
    linear_stress = []
    strain_deviation = []

    for i in range(0, len(strain)):
        if strain[i] < Split_Point:
            linear_strain.append(strain[i])
            linear_stress.append(stress[i])
    
    linear_strain = np.array(linear_strain)
    linear_stress = np.array(linear_stress)

    linear_strain = linear_strain[:,np.newaxis]
           
    E_trial,r,_,_ = np.linalg.lstsq(linear_strain,linear_stress)    # Finding Young's Modulus by forcing to zero intercept
    
    # Calculating mean strain deviation for the linear_stress, linear_strain data
    
    for i in range(0,len(linear_strain)):
        strain_deviation.append((linear_strain[i] - linear_stress[i]/E_trial))
        
    mean_strain_deviation = np.mean(strain_deviation)
    
    # Filtering out the stress, strain data with strain deviation within bounds 
    # of mean strain deviation +/- Extensometer least count
            
    filtered_stress =[]
    
    for i in range(0,len(strain)):
            if mean_strain_deviation-0.0005 < (strain[i]- (stress[i]/E_trial)) < mean_strain_deviation + 0.0005 :
                filtered_stress.append(stress[i])
    break_stress = np.max(filtered_stress)
    return(break_stress)
print('done')

done


# Finding Mechanical Properties 

In [147]:
# Function to calculate Mechanical Properties

def Mechanical_Properties(stress, strain, split_stress):

    #Calculating Young's Modulus from raw data ey_strain

    #Filtering out the linear potion of the ey_strain and stress data to find the slope

    linear_strain = []
    linear_stress = []

    for i in range(0, len(stress)):
        if stress[i] < split_stress:
            linear_strain.append(strain[i])
            linear_stress.append(stress[i])

    # Filtering out the linear potion of the ey_strain and stress data to fit a polynomial

    nonlinear_strain =[]
    nonlinear_stress =[]

    for i in range(0,len(stress)):
        if stress[i] > split_stress:
            nonlinear_strain.append(strain[i])
            nonlinear_stress.append(stress[i])
            
    linear_strain = np.array(linear_strain)
    linear_stress = np.array(linear_stress)

#     linear_strain = linear_strain[:,np.newaxis]

#     E,r,_,_ = np.linalg.lstsq(linear_strain,linear_stress)
    E,c,r,_,_ = stats.linregress(linear_strain,linear_stress)
    

    #Calculating R^2 value
    K = len(linear_stress)
    r_squared =  1 - r /(K * linear_stress.var())
    r_squared = round(r_squared,5)

    #Calculating coefficient of variation
    V = 100*np.sqrt(((1/r_squared)-1)/(K-2))

    
    
    #Fitting a polynomial curve to the nonlinear portion of stress-strain curve
    p = np.polyfit(nonlinear_strain,nonlinear_stress,8)
    p1d = np.poly1d(p)
    
    
    #Finding Ultimate Stress and Strain
    ucs = np.amax(stress)
    ucstrain = np.amax(strain)

    # To find yield stress from initially filtered data of Engineering strain vs Engineering stress
    # This works by finding intersection point of the straight line and polynomial

    x = np.linspace(0.002,0.05,100000)

    for i in range(0,len(x)):
        if (E*(x[i] -0.002)) > (p1d(x[i])):
            yieldstrain = x[i]
            yieldstress = p1d(x[i])
            break 


    #Caculating ductility from filtered data ey_strain
        
    ductility = (ucstrain - yieldstrain)*100
    ductility = round(ductility,2)
    return(round(yieldstress,2),round(ucs,2),int(E),round(ductility,2),round(r_squared,5),K,round(V,5))
    
print("done")


done


In [148]:
#Mechanical properties in different processing conditions

split_stress = 300  # Passing uniaxial strain, uniaxial stress
print(split_stress)

[y,ucs,E,d,R2,K,V] = Mechanical_Properties(Stress,Strain,split_stress)


dir = 'Results'
if not os.path.exists(dir):
    os.makedirs(dir)
else:
    shutil.rmtree(dir)           #removes all the subdirectories!
    os.makedirs(dir)

os.chdir('Results')



Data = pd.DataFrame({'Strain':Strain,'Stress':Stress},columns = ['Strain','Stress'])

Results = pd.DataFrame({'Result Type':['Uniaxial'],'Yield Strength(Mpa)':[y],'Ultimate Strength(Mpa)':[ucs],
                        'Youngs Modulus(Gpa)':[E/1000],'Ductility':[d],'R^2':[R2],'Data Points':[K],'CoV(%)':[V]},
                        columns = ['Result Type','Yield Strength(Mpa)','Ultimate Strength(Mpa)','Youngs Modulus(Gpa)',
                                   'Ductility','R^2','Data Points','CoV(%)'])

Results.to_csv('Results.csv')
Data.to_csv('Data.csv')

print(Results)


300
  Result Type  Yield Strength(Mpa)  Ultimate Strength(Mpa)  \
0    Uniaxial               424.68                  680.08   

   Youngs Modulus(Gpa)  Ductility      R^2  Data Points  CoV(%)  
0                   80      11.61  0.99999           10  0.1118  


In [149]:
# Plotting 

# To draw straight line in graph

b = np.linspace(0,0.01,1000)
g = np.linspace(0.002,0.015,2000)

a = E*b
c = E*(g - 0.002)


#plt.clf()

plt.figure(figsize = (20,15))
plt.rc('xtick', labelsize=20) 
plt.rc('ytick', labelsize=20)
plt.plot(Strain, Stress, 'k.')
plt.plot(b,a,'-')
plt.plot(g,c,'-')
plt.xlabel('Engineering Strain (mm/mm)', fontsize = 20)
plt.ylabel('Engineering Stress (Mpa)', fontsize = 20)
plt.xlim([0,Strain.max()+0.1])
plt.ylim([0,Stress.max()+200])
plt.savefig('Eng_Strain_Stress.tiff')


plt.show()
